In [1]:
import torch
import torch.nn as nn
from transformers import DistilBertConfig, DistilBertForSequenceClassification, AutoTokenizer
import pandas as pd
from torch.utils.data.dataloader import DataLoader


In [2]:
is_port_in_use(8080)  # Example usage, check if port 8080 is in use

False

In [8]:
z = s.split("</a>")[-1]

In [10]:
z.split("</a>")[-1]

' Tech layoffs  Google Microsoft and more continue to fire employees'

In [7]:
import numpy as np
n = np.random.random((5,2))
print(n)

[[0.94171782 0.99603348]
 [0.09840676 0.16916325]
 [0.08900746 0.34375377]
 [0.57314964 0.8889423 ]
 [0.09393409 0.66382003]]


In [17]:
df = pd.DataFrame(n,columns=['Column1', 'Column2'])
# df.reset_index('Column1')
df

,Column1,Column2
0,0.941718,0.996033
1,0.098407,0.169163
2,0.089007,0.343754
3,0.573150,0.888942
4,0.093934,0.663820


In [2]:
label_map = {'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4}
reverse_map = {0:'business', 1:'entertainment', 2:'politics', 3:'sport', 4:'tech'}

In [ ]:
class CustomCNN(nn.Module):
    
    def _init_(self,num_filters,hidden_size,filter_size=3,num_classes=10,activation_function='ReLU',n_blocks=5,filter_organisation_factor=2,dropout=0.4,batch_norm=True):
        """
        Custom CNN model for image classification.
        Args:
            num_filters (int): Number of filters in the first convolutional layer.
            size_filter (int): Size of the convolutional filters.
            num_classes (int): Number of output classes.
            activation (str): Activation function to use .
            n_blocks (int): Number of convolutional blocks.
        """
        super(CustomCNN,self)._init_()
        self.conv = nn.ModuleList()
        self.activation = {"ReLU": nn.ReLU(), "GeLU": nn.GELU(), "SiLU": nn.SiLU(), "Mish": nn.Mish()}
        in_channel = 3
        # convolutional blocks
        for i in range(n_blocks):
            in_channel = num_filters*(filter_organisation_factor**(i-1)) if i>0 else 3
            in_channel = int(in_channel)
            numberOfFilters = num_filters*(filter_organisation_factor**(i))
            numberOfFilters = int(numberOfFilters)
            # start of the convolutional block
            # convolution layer
            self.conv.append(nn.Conv2d(in_channels = in_channel, out_channels=numberOfFilters, kernel_size=filter_size, stride=1))
            # batch normalization layer
            if batch_norm:
                self.conv.append(nn.BatchNorm2d(numberOfFilters))
            # activation layer
            self.conv.append(self.activation[activation_function])
            # max pooling layer
            self.conv.append(nn.MaxPool2d(kernel_size=2, stride=2))
        # flattening last layer
        self.conv.append(nn.AdaptiveAvgPool2d(1))
        self.conv.append(nn.Flatten())
        # fully connected layer
        in_channel = num_filters*(filter_organisation_factor**(n_blocks-1))
        in_channel = int(in_channel)    
        self.conv.append(nn.Linear(in_channel, hidden_size))
        self.conv.append(self.activation['GeLU'])
        # dropout layer
        self.conv.append(nn.Dropout(dropout))
        self.conv.append(nn.Linear(hidden_size, num_classes))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        for layer in self.conv:
            x = layer(x)
        return x
    
    def train_model(self, 
                   train_loader: DataLoader, 
                   val_loader: DataLoader, 
                   epochs: int, 
                   learning_rate: float, 
                   device: torch.device,
                   test_loader: DataLoader = None,
                   criterion: nn.Module = nn.CrossEntropyLoss(),
                   optimizer_class: optim.Optimizer = optim.Adam):
        """
        Train the model with accuracy evaluation.
        Args:
            train_loader (DataLoader): DataLoader for training data.
            val_loader (DataLoader): DataLoader for validation data.
            epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimizer.
            device (torch.device): Device to train on ('cuda' or 'cpu').
            criterion (nn.Module): Loss function.
            optimizer_class (optim.Optimizer): Optimizer class (e.g., Adam, SGD).
        Returns:
            Dict[str, List[float]]: Dictionary containing training/validation losses and accuracies.
        """
        self.to(device)
        optimizer = optimizer_class(self.parameters(), lr=learning_rate)
        
        history = {
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_acc':[]
        }
        
        for epoch in range(epochs):
            self.train()
            running_loss = 0.0
            running_correct = 0
            total_samples = 0
            
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                batch_size = inputs.size(0)
                total_samples += batch_size
                
                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item() * batch_size
                running_correct += (outputs.argmax(1) == labels).sum().item()
 
                del loss
                del inputs
            # Calculate training metrics
            epoch_train_loss = running_loss / total_samples
            epoch_train_acc = running_correct / total_samples
            history['train_loss'].append(epoch_train_loss)
            history['train_acc'].append(epoch_train_acc)
            
            # Validation
            self.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0
            test_correct = 0
            test_total = 0
            
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    batch_size = inputs.size(0)
                    val_total += batch_size
                    
                    outputs = self(inputs)
                    loss = criterion(outputs, labels)
                    
                    val_loss += loss.item() * batch_size
                    val_correct += (outputs.argmax(1) == labels).sum().item()
                    del loss
                    del inputs
            if test_loader is not None:
                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        batch_size = inputs.size(0)
                        test_total += batch_size
                        
                        outputs = self(inputs)
                        loss = criterion(outputs, labels)
                        
                        test_correct += (outputs.argmax(1) == labels).sum().item()
                        del loss
                        del inputs
            # Calculate validation metrics
            epoch_val_loss = val_loss / val_total
            epoch_val_acc = val_correct / val_total
            epoch_test_acc = test_correct / test_total if test_loader is not None else None
            history['val_loss'].append(epoch_val_loss)
            history['val_acc'].append(epoch_val_acc)
            history['test_acc'].append(epoch_test_acc)
            
            # Print metrics
            print(f'Epoch {epoch+1}/{epochs}: '
                  f'Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.4f} | '
                  f'Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.4f}')
        
 
        return history
    
    def predict(self, 
                test_loader: DataLoader, 
                device: torch.device) :
        """
        Make predictions on test data.
        Args:
            test_loader (DataLoader): DataLoader for test data.
            device (torch.device): Device to use for prediction.
        Returns:
            Tuple[torch.Tensor, torch.Tensor]: Predictions and true labels.
        """
        self.eval()
        self.to(device)
        
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(device)
                outputs = self(inputs)
                _, preds = torch.max(outputs, 1)
                
                all_preds.append(preds.cpu())
                all_labels.append(labels.cpu())
        
        return torch.cat(all_preds), torch.cat(all_labels)

def transform_image(dataAugmentation=False):
    if dataAugmentation:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomRotation(15),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
        return transform
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    return transform

def data_loader(data_dir, batch_size, dataAugmentation, num_workers=3):
    # Load the full training dataset
    full_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'train'), transform=transform_image(dataAugmentation=dataAugmentation))
    targets = full_dataset.targets  # class labels for stratification

    # Stratified split: 80% train, 20% val
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    train_idx, val_idx = next(sss.split(full_dataset.samples, targets))

    # Subsets
    train_dataset = Subset(full_dataset, train_idx)
    val_dataset = Subset(full_dataset, val_idx)

    # Override transform for val set (no augmentation)
    val_dataset.dataset.transform = transform_image(dataAugmentation=False)

    # Test dataset
    test_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'val'), transform=transform_image(dataAugmentation=False))

    # Data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    return train_loader, val_loader, test_loader

In [4]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, DistilBertConfig, DistilBertForSequenceClassification

# Load model in half precision
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load tokenizer and model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Custom config (5 output classes + FP16 if CUDA)
config = DistilBertConfig.from_pretrained(
    model_name,
    num_labels=5,  # Your 5 classes
    torch_dtype=torch.float16 if device.type == 'cuda' else torch.float32
)

# Load pretrained model
model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    config=config,
    ignore_mismatched_sizes=True
).to(device)

# ====== KEY MODIFICATION ======
# 1. Add a new 256-unit layer before the classifier
model.classifier = nn.Sequential(
    nn.Linear(768, 512),  # New layer (768 -> 256)
    nn.GELU(),            # Activation
    nn.Linear(512, 5)     # Final classifier (256 -> 5 classes)
).to(device)

# 2. Freeze all layers EXCEPT the new classifier
for param in model.parameters():
    param.requires_grad = False  # Freeze entire model

# Unfreeze only the classifier
for param in model.classifier.parameters():
    param.requires_grad = True

# Verify architecture
print(model.classifier)
# Should show:
# Sequential(
#   (0): Linear(in_features=768, out_features=256)
#   (1): ReLU()
#   (2): Linear(in_features=256, out_features=5)
# )

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sequential(
  (0): Linear(in_features=768, out_features=512, bias=True)
  (1): GELU(approximate='none')
  (2): Linear(in_features=512, out_features=5, bias=True)
)


In [5]:
model.classifier

Sequential(
  (0): Linear(in_features=768, out_features=512, bias=True)
  (1): GELU(approximate='none')
  (2): Linear(in_features=512, out_features=5, bias=True)
)

In [6]:
device

device(type='cuda', index=0)

In [ ]:
class NewDataset(torch.utils.data.Dataset):
    def __init__(self,data):
        self.data = data.reset_index(drop=True)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = str(self.data.iloc[index]['Text'])  # Ensure string type
        label = int(self.data.iloc[index]['Category'])  # Ensure integer
        return {'text': text, 'label': label}

def data_loader(data,batch_size=32):
    """
    Load the data and create DataLoader objects for training and validation.
    """
    # Load your dataset here
    train = data.sample(frac=0.8, random_state=42)
    val = data.drop(train.index)
    train_dataset = NewDataset(train)
    val_dataset = NewDataset(val)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

def train_model(model,data,lr,batch_size,num_epoch):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    # criterion = nn.CrossEntropyLoss()
    train_loader, val_loader = data_loader(data=data,batch_size=batch_size)
    total_cnt = 0
    correct_cnt = 0
    for epoch in range(num_epoch):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs = tokenizer(batch['text'], return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            labels = torch.tensor(batch['label'].tolist()).to(device)
            outputs = model(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                labels = labels
            )
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            correct_cnt += (torch.argmax(outputs.logits, dim=1) == labels).sum().item()
            total_cnt += labels.size(0)
        print(f"Epoch {epoch+1}/{num_epoch}, Loss: {loss.item()}, accuracy: {correct_cnt / total_cnt}")
        with torch.no_grad():
            model.eval()
            val_loss = 0
            total_cnt = 0
            correct_cnt = 0
            for batch in val_loader:
                inputs = tokenizer(batch['text'], return_tensors="pt", padding=True, truncation=True).to(device)
                labels = torch.tensor(batch['label'].tolist()).to(device)
                outputs = model(**inputs, labels=labels)
                val_loss += outputs.loss.item()
                correct_cnt += (torch.argmax(outputs.logits, dim=1) == labels).sum().item()
                total_cnt += labels.size(0)
            val_loss /= len(val_loader)
            accuracy = correct_cnt / total_cnt
            print(f"Epoch {epoch+1}/{num_epoch}, Validation Loss: {val_loss}, Accuracy: {accuracy}")
    
def predict(model,news:list):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(news, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    model.eval()
    outputs = model(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask']
    )
    label = torch.argmax(outputs.logits, dim=1)
    label = [reverse_map[label.data[i].item()] for i in range(len(label))]
    return label

In [ ]:
'<a href=""https://www.indiatoday.in/world/video/china-urges-us-cancel-reciprocal-tariffs-donald-trump-xi-jinping-2708501-2025-04-14""> <img align=""left"" alt="""" border=""0"" height=""180"" hspace=""2"" src=""https://akm-img-a-in.tosshub.com/indiatoday/images/video/202504/us-china-tariff-war-133601625-16x9_0.jpg?VersionId=2cszWtvdpM8FvhA9z5MTDbfWTnHDgvRv"" width=""305"" /> </a> China urges US to cancel reciprocal tariffs calls for mutual respect'

In [14]:
data = pd.read_csv("BBC News Train.csv")
data = pd.concat([data["Text"],data["Category"]],axis=1)
data['Category'] = data['Category'].map(label_map)
train_model(model,data=data,lr=1e-2,batch_size=32,num_epoch=5)

Epoch 1/5, Loss: 0.6392111778259277, accuracy: 0.7139261744966443
Epoch 1/5, Validation Loss: 0.5903841435909272, Accuracy: 0.7818791946308725
Epoch 2/5, Loss: 0.6005663275718689, accuracy: 0.7644295302013423
Epoch 2/5, Validation Loss: 0.5940935492515564, Accuracy: 0.7785234899328859
Epoch 3/5, Loss: 0.883898138999939, accuracy: 0.7919463087248322
Epoch 3/5, Validation Loss: 0.518662104010582, Accuracy: 0.8087248322147651
Epoch 4/5, Loss: 0.1084352433681488, accuracy: 0.8
Epoch 4/5, Validation Loss: 0.4229733467102051, Accuracy: 0.8590604026845637
Epoch 5/5, Loss: 1.3128069639205933, accuracy: 0.8033557046979866
Epoch 5/5, Validation Loss: 0.3608491882681847, Accuracy: 0.8624161073825504


In [ ]:
z = predict(model,["In a significant boost to India's futuristic military capability development, the Defence and Research Development Organisation (DRDO) Sunday showcased a 30-kilowatt laser-based weapon system designed to take down helicopters, swarm drones, and radars.",
                   "It was Jasprit Bumrah's magic which saw Mumbai Indians come out on top, beating Delhi Capitals by 12 runs in their IPL 2025 fixture, at the Arun Jaitley Stadium in New Delhi on Sunday. Chasing 206 runs, DC had a sudden collapse in the 19th over as three run-outs in the 19th over saw Ashutosh Sharma (17), Kuldeep Yadav (1) and Mohit Sharma (0) lose their wickets in an error-prone end. In the run-chase, Delhi were dealt an early blow as Deepak Chahar removed opener Jake Fraser-McGurk for a golden duck in the first ball of the opening over. But since then, Abishek Porel and Karun Nair have stitched together a partnership, which has already gone past 100 runs, and the latter has also registered a 22-ball fifty",
                   "Meet Laila Faisal, the rumoured girlfriend of star Indian batter Abhishek Sharma. A London-educated entrepreneur, she runs a luxury fashion brand and is making headlines for her glamorous style and growing connection with the IPL 2025 sensation.",
                   "IPL 2025 star Abhishek Sharma has been turning heads not only with his cricketing prowess but also due to his rumored relationship with fashion entrepreneur Laila Faisal. Whispers about the duo intensified when Laila shared a heartfelt Instagram story celebrating Abhishek’s stunning 141-run knock against Punjab Kings — a performance hailed as one of the best this season. While the couple hasn't officially confirmed their relationship, their public chemistry is fueling ongoing buzz.",
                   "As the trade war between the US and China escalates, Beijing has suspended the export of several critical rare earth elements, metals and magnets, threatening to choke off supplies to the West",
                   '<a href=""https://www.indiatoday.in/trending-news/story/uk-woman-arrested-for-stealing-daughters-ipads-she-took-away-as-punishment-2708348-2025-04-13""> <img align=""left"" alt="""" border=""0"" height=""180"" hspace=""2"" src=""https://akm-img-a-in.tosshub.com/indiatoday/images/story/202504/uk-woman-arrested-for-stealing-her-daughters-ipads-133251723-16x9_0.jpg?VersionId=Rp60sqsQTuXSX24nsQhPYlilR71_ybeD"" width=""305"" /> </a> UK woman arrested for stealing daughters iPads she took away as punishment',
                   '<a href=""https://www.indiatoday.in/world/video/china-urges-us-cancel-reciprocal-tariffs-donald-trump-xi-jinping-2708501-2025-04-14""> <img align=""left"" alt="""" border=""0"" height=""180"" hspace=""2"" src=""https://akm-img-a-in.tosshub.com/indiatoday/images/video/202504/us-china-tariff-war-133601625-16x9_0.jpg?VersionId=2cszWtvdpM8FvhA9z5MTDbfWTnHDgvRv"" width=""305"" /> </a> China urges US to cancel reciprocal tariffs calls for mutual respect',
                   "Make Amit Shah apologise on Ambedkar, demands Chamala"])

In [32]:
# loading library
import pickle
with open('model_pkl', 'wb') as files:
    pickle.dump(model, files)

In [38]:
with open('model_pkl' , 'rb') as f:
    model1 = pickle.load(f)

In [ ]:
z = predict(model1,["In a significant boost to India's futuristic military capability development, the Defence and Research Development Organisation (DRDO) Sunday showcased a 30-kilowatt laser-based weapon system designed to take down helicopters, swarm drones, and radars.",
                   "It was Jasprit Bumrah's magic which saw Mumbai Indians come out on top, beating Delhi Capitals by 12 runs in their IPL 2025 fixture, at the Arun Jaitley Stadium in New Delhi on Sunday. Chasing 206 runs, DC had a sudden collapse in the 19th over as three run-outs in the 19th over saw Ashutosh Sharma (17), Kuldeep Yadav (1) and Mohit Sharma (0) lose their wickets in an error-prone end. In the run-chase, Delhi were dealt an early blow as Deepak Chahar removed opener Jake Fraser-McGurk for a golden duck in the first ball of the opening over. But since then, Abishek Porel and Karun Nair have stitched together a partnership, which has already gone past 100 runs, and the latter has also registered a 22-ball fifty",
                   "Meet Laila Faisal, the rumoured girlfriend of star Indian batter Abhishek Sharma. A London-educated entrepreneur, she runs a luxury fashion brand and is making headlines for her glamorous style and growing connection with the IPL 2025 sensation.",
                   "IPL 2025 star Abhishek Sharma has been turning heads not only with his cricketing prowess but also due to his rumored relationship with fashion entrepreneur Laila Faisal. Whispers about the duo intensified when Laila shared a heartfelt Instagram story celebrating Abhishek’s stunning 141-run knock against Punjab Kings — a performance hailed as one of the best this season. While the couple hasn't officially confirmed their relationship, their public chemistry is fueling ongoing buzz.",
                   "As the trade war between the US and China escalates, Beijing has suspended the export of several critical rare earth elements, metals and magnets, threatening to choke off supplies to the West",
                   '<a href=""https://www.indiatoday.in/trending-news/story/uk-woman-arrested-for-stealing-daughters-ipads-she-took-away-as-punishment-2708348-2025-04-13""> <img align=""left"" alt="""" border=""0"" height=""180"" hspace=""2"" src=""https://akm-img-a-in.tosshub.com/indiatoday/images/story/202504/uk-woman-arrested-for-stealing-her-daughters-ipads-133251723-16x9_0.jpg?VersionId=Rp60sqsQTuXSX24nsQhPYlilR71_ybeD"" width=""305"" /> </a> UK woman arrested for stealing daughters iPads she took away as punishment',
                   '<a href=""https://www.indiatoday.in/world/video/china-urges-us-cancel-reciprocal-tariffs-donald-trump-xi-jinping-2708501-2025-04-14""> <img align=""left"" alt="""" border=""0"" height=""180"" hspace=""2"" src=""https://akm-img-a-in.tosshub.com/indiatoday/images/video/202504/us-china-tariff-war-133601625-16x9_0.jpg?VersionId=2cszWtvdpM8FvhA9z5MTDbfWTnHDgvRv"" width=""305"" /> </a> China urges US to cancel reciprocal tariffs calls for mutual respect',
                   "Make Amit Shah apologise on Ambedkar, demands Chamala"])

In [ ]:
model1.pre

In [ ]:
# next(z)['text'].tolist()
f = tokenizer(next(z)['text'], return_tensors="pt", padding=True, truncation=True)

In [ ]:
torch.tensor(next(z)['label'].tolist()).to(device)

tensor([4, 0, 2, 3, 0, 1, 1, 0, 0, 2, 2, 2, 4, 4, 1, 3, 3, 2, 2, 2, 1, 3, 4, 4,
        0, 1, 3, 2, 3, 1, 0, 3], device='cuda:0')

In [ ]:
for labels in next(z):
    # print(txt)
    print(labels)
    # brea

text
label


In [ ]:
# train_loader, val_loader = data_loader()

# Iterate through batches
for batch_idx, batch_data in enumerate(iter(train_loader)):
    texts, labels = batch_data  # Unpack your batch
    print(f"Batch {batch_idx}:")
    print("Text samples:", texts)  # First 2 texts in batch
    print("Labels:", labels)      # Corresponding labels
    break  # Remove this to process all batches

Batch 0:
Text samples: text
Labels: label


In [ ]:
data = pd.read_csv("BBC News Train.csv")
train = data.sample(frac=0.8, random_state=42)
val = data.drop(train.index)

In [ ]:
z = pd.concat([train["Text"],train["Category"]],axis=1)

In [ ]:
z['Category'] = z['Category'].map(label_map)

In [ ]:
type(z['Category'].iloc[0])

numpy.int64

In [ ]:
pd.factorize(z['Category'])

(array([0, 1, 1, ..., 0, 0, 2], shape=(1192,)),
 Index([1, 2, 4, 0, 3], dtype='int64'))